<center><h1 style="color:#D4AF37"> AEMR Business Analysis </h1>

<img src = "https://images.squarespace-cdn.com/content/v1/551972d8e4b0d571edc2e2c8/8f3abd33-0916-4415-9b24-593eea1d1e3c/Screen+Shot+2022-04-20+at+12.41.24+PM.png">

  


<h1 style="color:#D4AF37"> What's the Business Problem? </h1>

The American Energy Market Regulator (AEMR) is responsible for looking after the
United States of America’s domestic energy network. The regulator’s responsibility is to
ensure that America’s energy network remains reliable with minimal disruptions, which
are known as outages.

There are four key types of outages:

● Consequential

● Forced

● Opportunistic

● Planned

Recently, the AEMR management team has been increasingly aware of a large number
of energy providers that submitted outages over the 2016 and 2017 calendar years. The
management team has expressed a desire to have the following two areas of concern
addressed:

<b> A) Energy Stability and Market Outages
    <p>
B) Energy Losses and Market Reliability </b>`

**Setup: Load AEMR Database into Environment**

In [2]:
import requests
from IPython.core.magic import register_line_magic
from IPython.display import HTML
import sqlite3

@register_line_magic
def load_sqlite_db(url):
    response = requests.get(url)

    if response.status_code == 200:
        with open('temp_db_file.db', 'wb') as file:
            file.write(response.content)
        print('SQLite database file downloaded successfully.')
    else:
        print('Failed to download the SQLite database file.')

sqlite_db_url = 'https://raw.githubusercontent.com/chrishuisb1990/practice_datasets/main/AEMR.db'

%load_sqlite_db $sqlite_db_url

%load_ext sql

%sql sqlite:///temp_db_file.db
%config SqlMagic.style = '_DEPRECATED_DEFAULT'


SQLite database file downloaded successfully.


<h1 style="color:#D4AF37"> Part I. Energy Stability & Market Outages </h1>

**Context:**
<p>
Energy stability is a top priority for the AEMR. To maintain energy security and market reliability, it is critical to understand the patterns and severity of outage events. Key questions driving this analysis include:

<p>
<b>
<li>  What are the most common types of outages, and how long do they typically last?
<li>  How frequently do these outages occur?
<li>  Are certain energy providers consistently more prone to outages, signaling potential systemic reliability risks?
    
<p>


# <h3 style="color:#D4AF37"> Business Question One </h3>
<b> How do outage volumes vary by outage type (Forced, Scheduled, Opportunistic, Consequential) across 2016 and 2017, and what trends or reliability risks emerge over time? </b>



In [4]:
%%sql
SELECT
    COUNT(*) AS total_number_outages,
    outage_reason, year
FROM AEMR_Outage_Table
WHERE status = 'Approved'
    AND year IN (2016, 2017)
GROUP BY outage_reason, year
ORDER BY outage_reason, year;

 * sqlite:///temp_db_file.db
Done.


total_number_outages,Outage_Reason,Year
181,Consequential,2016
127,Consequential,2017
1264,Forced,2016
1622,Forced,2017
106,Opportunistic Maintenance (Planned),2016
102,Opportunistic Maintenance (Planned),2017
380,Scheduled (Planned),2016
320,Scheduled (Planned),2017


**Insight:** Between 2016 and 2017, **forced outages rose by 28%**, signaling increased unplanned disruptions and potential reliability concerns. In contrast, scheduled outages fell by 16%, and consequential outages dropped by 30%, suggesting improved containment and planning. Opportunistic maintenance remained stable, reflecting limited proactive flexibility. Overall, forced outages increased, it might suggest declining infrastructure reliability or external factors affecting stability.

<h3 style="color:#D4AF37"> Business Question Two </h3>



<b>What are the monthly trends in approved outage events across 2016 and 2017, and which outage types are increasing over time? </b>




In [5]:
%%sql
select
    year, month,
    COUNT(*) AS total_number_outages
from AEMR_Outage_Table
where status = 'Approved'
    AND year in (2016,2017)
group by year, month
order by year, month, total_number_outages desc;


 * sqlite:///temp_db_file.db
Done.


Year,Month,total_number_outages
2016,1,191
2016,2,227
2016,3,136
2016,4,134
2016,5,174
2016,6,168
2016,7,147
2016,8,194
2016,9,124
2016,10,124


**Insight:**
The analysis reveals clear seasonal spikes in outage events, with the second half of the year (July to December) showing consistently higher volumes. This trend may be driven by environmental conditions (e.g., heatwaves or storms) or operational constraints such as delayed maintenance.

While one outage type shows a notable increase in frequency from 2016 to 2017, frequency alone doesn’t fully capture its impact. It’s equally important to consider **outage duration**—as short interruptions may be negligible, but prolonged outages pose a significant threat to energy reliability and supply security.

<h3 style="color:#D4AF37"> Business Question Three </h3>



    
 **Which participants are responsible for the most frequent and prolonged approved outages by type, and what operational risks do they pose?**





In [6]:
%%sql
SELECT
    Participant_Code,
    Outage_Reason,
    Year,
    COUNT(*) AS total_number_outages,
    ROUND(AVG(ABS(JULIANDAY(end_time) - JULIANDAY(start_time))),2) AS avg_outage_duration_days
FROM AEMR_Outage_Table
WHERE status = 'Approved'
GROUP BY 1,2,3
ORDER BY total_number_outages desc;


 * sqlite:///temp_db_file.db
Done.


Participant_Code,Outage_Reason,Year,total_number_outages,avg_outage_duration_days
AURICON,Forced,2017,490,0.07
GW,Forced,2016,317,0.38
GW,Forced,2017,227,1.06
AURICON,Forced,2016,208,0.07
AUXC,Forced,2016,206,0.08
MELK,Forced,2017,177,2.28
TRMOS,Forced,2017,172,0.42
MELK,Forced,2016,157,0.83
PUG,Forced,2017,135,0.25
AUXC,Forced,2017,120,0.02


**Insights: Participant-Level Outage Risk Overview (2016–2017)**

- **AURICON** reported the highest number of forced outages in both 2016 (208) and 2017 (490). Despite the high volume, the average outage duration was just **0.07 days**, indicating frequent but low-severity events. This suggests potential reliability issues tied to recurring minor disruptions or gaps in preventive maintenance.

- **GW** experienced a significant rise in outage duration—from **0.38 days in 2016** to **1.06 days in 2017**—alongside consistently high outage counts. This trend signals deteriorating recovery efficiency and a growing risk to operational continuity.

- **MELK** exhibited the most critical pattern, with both high outage frequency and the **longest durations**: **0.83 days in 2016** and **2.28 days in 2017**. This presents a major reliability concern and highlights the need for urgent infrastructure and process evaluation.

> While AURICON poses a high-frequency but low-impact risk, **GW and MELK represent more severe systemic threats** due to the compounding effect of recurring, prolonged outages. These entities should be prioritized for compliance reviews, root cause investigations, and mitigation planning.



<h3 style="color:#D4AF37"> Business Question Four </h3>

<b> Which participants can be classified as High, Medium, or Low Risk based on their average outage duration, and how can this segmentation inform operational decision-making?



Risk classification criteria
<b>
<li> High Risk - On average, the participant is unavailable for > 24 Hours (1 Day)</li>
<li> Medium Risk - On average, the participant is unavailable between 12 and 24 Hours </li>
<li> Low Risk - On average, the participant is unavailable for less than 12 Hours</li>
</b>

In [7]:
%%sql
with avg_duration_days as(
    SELECT
    Participant_Code,
    Outage_Reason,
    Year,
    COUNT(*) AS total_number_outages,
    ROUND(AVG(ABS(JULIANDAY(end_time) - JULIANDAY(start_time))),2) AS avg_outage_duration_days
FROM AEMR_Outage_Table
WHERE status = 'Approved'
GROUP BY Participant_Code,Outage_Reason,Year
ORDER BY total_number_outages desc)

SELECT *,
CASE
    WHEN avg_outage_duration_days > 1 then 'High Risk'
    WHEN avg_outage_duration_days between 0.5 and 1 then 'Medium Risk'
    WHEN avg_outage_duration_days between 0 and 0.5 then 'Low Risk'
    ELSE 'N/A'
    END as Risk_Classification
from avg_duration_days


 * sqlite:///temp_db_file.db
Done.


Participant_Code,Outage_Reason,Year,total_number_outages,avg_outage_duration_days,Risk_Classification
AURICON,Forced,2017,490,0.07,Low Risk
GW,Forced,2016,317,0.38,Low Risk
GW,Forced,2017,227,1.06,High Risk
AURICON,Forced,2016,208,0.07,Low Risk
AUXC,Forced,2016,206,0.08,Low Risk
MELK,Forced,2017,177,2.28,High Risk
TRMOS,Forced,2017,172,0.42,Low Risk
MELK,Forced,2016,157,0.83,Medium Risk
PUG,Forced,2017,135,0.25,Low Risk
AUXC,Forced,2017,120,0.02,Low Risk


**Insights: Risk-Based Participant Classification**

Using average outage duration as the classification criterion, participants were segmented into **High**, **Medium**, and **Low Risk** tiers:

**High Risk Participants**
These participants experienced **average outages > 1 day (24 hours)** and therefore pose the **greatest risk to operational stability**. Notably:

- **MELK**: Consistently appears across multiple outage types with durations exceeding 2 days (e.g., 2.28 in 2017 and 6.89 for Scheduled).
- **GW**: While showing a high volume of outages, it also recorded several Scheduled outages with durations of **4.43 days (2016)** and **2.65 days (2017)**.
- **AURICON**, **COLLGAR**, **KORL**, **PMC**, and **ENRG** also demonstrated prolonged scheduled or forced outages above the 1-day threshold.

These participants require **targeted audits**, **preventive infrastructure improvements**, and **closer compliance monitoring** to reduce downtime.

---

**Medium Risk Participants**
With average outage durations between **12 and 24 hours**, these participants are **moderate stability risks**, including:

- **PJRH**, **TSLA_MGT**, **MELK** (in specific years or outage types), **PUG**, and **AUXC**.
- Although their events are less severe than high-risk participants, these still have potential for compounding operational strain.

They should be **monitored proactively** to prevent escalation and reclassified if durations worsen over time.

---

**Low Risk Participants**
These participants maintain **short average outages (< 12 hours)** and are **operationally stable**:

- Includes high-volume entities like **AURICON** (Forced outages averaging 0.07 days) and **AUXC**, **TRMOS**, **DNHR**, among others.
- These facilities generally present **manageable and predictable outage patterns**, though volume should still be tracked to catch emerging issues.

They can serve as **benchmarks for operational excellence**, but should still adhere to preventive maintenance standards.



**Conclusion**

Risk-based segmentation provides a **data-driven framework** for prioritizing intervention strategies. By focusing resources on **High Risk participants** with long durations, AEMR can **improve grid resilience**, **reduce energy losses**, and **optimize system-wide reliability**. **Medium Risk participants** should be monitored closely, while **Low Risk participants** can maintain current protocols with periodic review.


While outage types such as **Consequential**, **Opportunistic**, and **Planned** contribute to overall operational metrics, only **Forced outages** are **unplanned** and pose a direct risk to **grid reliability and energy security**. Including all outage types equally in the risk model may dilute focus from the most critical threats.

To address this, the revised framework focuses solely on **Forced outages**, using a **dual-criteria approach** that incorporates both **average outage duration** and **event frequency** to more accurately assess operational risk.


Risk Classification (Forced Outages Only)

- **High Risk**  
  - Average duration > 24 hours  OR
  - More than 20 outage events  

- **Medium Risk**  
  - Average duration between 12–24 hours OR
  - 10–20 outage events  

- **Low Risk**  
  - Average duration < 12 hours OR
  - Fewer than 10 outage events  

This refined model ensures focus remains on **critical unplanned outages**, enabling more effective monitoring, prioritization, and mitigation strategies.



<h3 style="color:#D4AF37"> Business Question Five </h3>

<b> Which participants pose the greatest operational risk based on the frequency and average duration (in days) of their forced outages?



In [8]:
%%sql
with avg_duration_days as(
    SELECT
    Participant_Code,
    Outage_Reason,
    Year,
    COUNT(*) AS total_number_outages,
    ROUND(AVG(ABS(JULIANDAY(end_time) - JULIANDAY(start_time))),2) AS avg_outage_duration_days
FROM AEMR_Outage_Table
WHERE status = 'Approved'
GROUP BY Participant_Code,Outage_Reason,Year
ORDER BY total_number_outages desc)

SELECT *,
CASE
    WHEN avg_outage_duration_days > 1 AND Outage_Reason = 'Forced'
    OR total_number_outages > 20 then 'High Risk'
    WHEN avg_outage_duration_days between 0.5 and 1 AND Outage_Reason = 'Forced'
    OR total_number_outages between 10 and 20 then 'Medium Risk'
    WHEN avg_outage_duration_days between 0 and 0.5 AND Outage_Reason = 'Forced'
    OR total_number_outages > 10 then 'Low Risk'
    ELSE 'N/A'
    END as Risk_Classification
from avg_duration_days

 * sqlite:///temp_db_file.db
Done.


Participant_Code,Outage_Reason,Year,total_number_outages,avg_outage_duration_days,Risk_Classification
AURICON,Forced,2017,490,0.07,High Risk
GW,Forced,2016,317,0.38,High Risk
GW,Forced,2017,227,1.06,High Risk
AURICON,Forced,2016,208,0.07,High Risk
AUXC,Forced,2016,206,0.08,High Risk
MELK,Forced,2017,177,2.28,High Risk
TRMOS,Forced,2017,172,0.42,High Risk
MELK,Forced,2016,157,0.83,High Risk
PUG,Forced,2017,135,0.25,High Risk
AUXC,Forced,2017,120,0.02,High Risk


**Insights:**

- **AURICON, GW, MELK, TRMOS, and PUG** top the **High Risk** list due to frequent **forced outages**, despite many of their individual outages being short in duration.  
This pattern indicates **chronic operational instability** and highlights the need for **preventive maintenance** and **proactive monitoring**.

- **MELK** stands out for exhibiting both **high frequency** and **prolonged outage durations** (e.g., **2.28 days** in 2017), signaling **critical reliability concerns** that may require urgent intervention.

- Participants like **WGUTD** and **ENRG** are currently classified as **Low or Medium Risk** due to fewer or shorter outage durations, but they should still be **monitored over time** to detect any upward trends that may elevate their risk profile.


<h1 style="color:#D4AF37"> Part II. Energy Losses & Market Reliability </h1>

### Objective

As part of a broader initiative to evaluate system resilience, this analysis focuses on **forced outages** to understand their frequency, duration, and overall impact on energy reliability. Given that forced outages are **unplanned supply disruptions**, they present a significant risk to energy security—particularly if multiple providers are affected concurrently.

To support AEMR’s efforts in mitigating these risks, the analysis addresses the following key business questions:

- **What proportion of all outages in 2016 and 2017 were classified as Forced Outages?**
- **What was the average duration of a forced outage in each of those years, and has the duration increased over time?**
- **Which energy providers recorded the highest number of forced outages, and do any patterns indicate systemic reliability issues?**


    
<img src = "https://media.istockphoto.com/id/1281821795/photo/market-stock-graph-and-information-with-city-light-and-electricity-and-energy-facility-banner.jpg?s=612x612&w=0&k=20&c=RSN5LqeMW28HW10aA190_DWR5YJ5tG2wixHFPBV3uZE=">

<h3 style="color:#D4AF37"> Business Question Six </h3>

<b> What proportion of all approved outages were classified as Forced in 2016 and 2017, and which outage types showed notable increases in frequency over this period? </b>



In [9]:
%%sql
SELECT Year,Total_Outage_Events,Total_Count_Of_Forced_Events,
    ROUND(1.0 * Total_Count_Of_Forced_Events / Total_Outage_Events *100.00, 2) AS Pct_Outage_Forced
FROM (
    SELECT
        Year, Status,
        COUNT(*) AS Total_Outage_Events,
        SUM(CASE WHEN Outage_Reason = 'Forced' THEN 1 ELSE 0 END) AS Total_Count_Of_Forced_Events
        FROM AEMR_Outage_Table
        WHERE Status = 'Approved'
        GROUP BY YEAR) AS sub;


 * sqlite:///temp_db_file.db
Done.


Year,Total_Outage_Events,Total_Count_Of_Forced_Events,Pct_Outage_Forced
2016,1931,1264,65.46
2017,2171,1622,74.71


**Insight:**
Between 2016 and 2017, forced outages increased both in absolute count and as a share of total approved outages, rising from 65.5% to 74.7%. This notable shift highlights a growing operational reliance on reactive recovery rather than proactive maintenance.


It’s evident that Forced Outages pose a significant operational and financial risk to the energy grid. Unlike other outage types, forced events are unplanned, meaning energy commitments are broken, leading to immediate energy loss and potential market instability.

Additionally, a subset of energy participants consistently exhibits a disproportionately high number of forced outages, signaling recurring reliability issues.

To drive actionable insights, analysis can be broken down into two layers:

**Macro Analysis** – Measures the total duration of outages per participant to understand overall energy loss impact.

**Micro Analysis** – Examines the average outage duration, revealing whether a few large events are skewing the totals, or whether outages are persistently long throughout the year.

This layered approach helps me to distinguish between participants affected by isolated major incidents vs. those with systemic reliability issues, and ultimately informs regulatory prioritization and preventive intervention strategies.

<h3 style="color:#D4AF37"> Business Question Seven </h3>

**Which participants and facilities contributed most to overall energy loss due to outages, and how can we quantify the operational impact by year?**
    

In [10]:
%%sql
SELECT
 COUNT(*) as Total_Number_Outages,
 ROUND(SUM(ABS(JULIANDAY(end_time) - JULIANDAY(start_time))),2) as Total_Duration_In_Days,
 ROUND(SUM(Energy_Lost_MW),2) as Total_Energy_Lost, Outage_Reason, Participant_Code, Facility_Code, Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Outage_Reason, Participant_Code, Facility_Code, Year
ORDER BY Total_Duration_In_Days desc, Year;

 * sqlite:///temp_db_file.db
Done.


Total_Number_Outages,Total_Duration_In_Days,Total_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
70,482.58,7499.28,Scheduled (Planned),MELK,MELK_G7,2017
177,404.15,10285.4,Forced,MELK,MELK_G7,2017
85,392.25,9668.79,Scheduled (Planned),MELK,MELK_G7,2016
227,240.69,19326.56,Forced,GW,BW1_GREENWATERS_G2,2017
45,199.4,6450.0,Scheduled (Planned),GW,BW1_GREENWATERS_G2,2016
37,183.58,516.92,Scheduled (Planned),ENRG,ENRG_KALGOORLIE_GT3,2017
29,140.6,445.6,Scheduled (Planned),ENRG,ENRG_KALGOORLIE_GT3,2016
20,139.25,1685.6,Scheduled (Planned),KORL,KORL_GT3,2016
157,129.6,13771.07,Forced,MELK,MELK_G7,2016
317,120.6,15751.38,Forced,GW,BW1_GREENWATERS_G2,2016


**Insight:**

An analysis of outage data from **2016–2017** reveals that a **small subset of participants and facilities** account for a **disproportionate share of total energy loss**.

- Specifically, **GW’s `BW1_GREENWATERS_G2`** and **MELK’s `MELK_G7`** facilities consistently top the list due to:
  - **High outage frequency**, and
  - **Extended outage durations**, particularly from **forced outages**.

These patterns highlight **systemic reliability issues** at **critical nodes in the grid**, suggesting a need for:
- **Targeted maintenance investment**
- **Operational risk reviews**
- **Proactive resilience planning**


<h3 style="color:#D4AF37"> Business Question Eight </h3>

<b>Which participants and facilities experience the most severe forced outages on average, and how does this vary by year?


In [11]:
%%sql
SELECT
ROUND(AVG(ABS(JULIANDAY(end_time) - JULIANDAY(start_time))),2) as Avg_Duration_In_Days,
ROUND(AVG(Energy_Lost_MW),2) as Avg_Energy_Lost, Outage_Reason, Participant_Code, Facility_Code, Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved' and Outage_Reason = 'Forced'
GROUP BY Participant_Code, Facility_Code, Year
ORDER BY Avg_Energy_Lost desc, Year desc;


 * sqlite:///temp_db_file.db
Done.


Avg_Duration_In_Days,Avg_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
1.11,149.0,Forced,COLLGAR,COLLGAR_WF1,2016
0.04,141.21,Forced,PMC,PMC_AG,2017
0.49,131.78,Forced,PMC,PMC_AG,2016
0.83,87.71,Forced,MELK,MELK_G7,2016
1.06,85.14,Forced,GW,BW1_GREENWATERS_G2,2017
0.25,80.0,Forced,TSLA_MGT,TESLA_PICTON_G1,2016
0.38,76.23,Forced,KORL,KORL_GT3,2016
1.22,72.61,Forced,PJRH,PJRH_GT11,2016
0.84,67.21,Forced,PJRH,PJRH_GT11,2017
1.38,61.93,Forced,COLLGAR,COLLGAR_WF1,2017


**Insights**:

Analysis of forced outages from 2016–2017 reveals a concentration of severe events among a few key participants and facilities. Notably:

- **COLLGAR**, **PMC**, **MELK**, and **GW** consistently experienced high average energy losses per forced outage, indicating significant operational disruptions.  
- **MELK_G7** and **COLLGAR_WF1** stand out for combining both **long outage durations** (1–2.3 days) and **high energy loss**, reflecting systemic reliability issues.
- **PMC_AG**, while averaging very **short outage durations** (as low as 0.04 days), still contributed to high energy loss—suggesting high-capacity facilities where even brief outages carry major impact.
- **WGUTD** recorded the **longest single average outage** (3.44 days in 2017), but with comparatively lower energy lost, pointing to possibly smaller capacity or less critical infrastructure.
- Some participants such as **EUCT** and **DNHR** had minimal operational impact due to either low energy loss or isolated events.

These findings help prioritize where AEMR should focus reliability improvements—targeting high-loss facilities with frequent or prolonged forced outages.



# <h3 style="color:#D4AF37"> Business Question Nine </h3>

<b> Which participants and facilities contributed the most to total energy lost from forced outages during 2016–2017, and what proportion of their total energy loss was due to forced events?


In [12]:
%%sql

SELECT
 ROUND(AVG(Energy_Lost_MW),2) as Avg_Energy_Lost,
 ROUND(SUM(Energy_Lost_MW),2) as Total_Energy_Lost,
 ROUND(SUM(Energy_Lost_MW)/(SELECT SUM(Energy_Lost_MW)
                            FROM AEMR_Outage_Table
                            WHERE Status = 'Approved' and Outage_Reason = 'Forced')*100.0,2) as Pct_Energy_Loss,
       Facility_Code, Participant_Code, Outage_Reason, Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved' and Outage_Reason = 'Forced'
GROUP BY Facility_Code, Participant_Code, Year
ORDER BY Total_Energy_Lost desc, Year desc;

 * sqlite:///temp_db_file.db
Done.


Avg_Energy_Lost,Total_Energy_Lost,Pct_Energy_Loss,Facility_Code,Participant_Code,Outage_Reason,Year
44.16,21639.55,14.21,AURICON_PNJ_U1,AURICON,Forced,2017
85.14,19326.56,12.69,BW1_GREENWATERS_G2,GW,Forced,2017
49.69,15751.38,10.34,BW1_GREENWATERS_G2,GW,Forced,2016
87.71,13771.07,9.04,MELK_G7,MELK,Forced,2016
51.42,10696.28,7.02,AURICON_PNJ_U1,AURICON,Forced,2016
58.11,10285.4,6.75,MELK_G7,MELK,Forced,2017
131.78,9093.09,5.97,PMC_AG,PMC,Forced,2016
72.61,5881.52,3.86,PJRH_GT11,PJRH,Forced,2016
141.21,5648.44,3.71,PMC_AG,PMC,Forced,2017
29.17,5016.67,3.29,TIWEST_COG1,TRMOS,Forced,2017


**Insight:**  
Between 2016 and 2017, forced outages led to significant energy losses, with a small group of participants and facilities contributing a disproportionately high share.

- **AURICON_PNJ_U1 (AURICON)** reported the highest total energy lost from forced outages, with over **21,600 MWh** lost in 2017—accounting for **14.2%** of all forced-outage-related energy loss that year.

- **BW1_GREENWATERS_G2 (GW)** and **MELK_G7 (MELK)** also consistently ranked among the top contributors across both years, indicating **ongoing reliability concerns**.

- **PMC_AG (PMC),** while experiencing shorter outage durations, recorded substantial energy loss due to its **high-capacity generation**, totaling **over 14,700 MWh** lost across 2016 and 2017.

> **Overall**, the top 5 facilities accounted for **more than 50%** of total energy lost from forced outages, underscoring the need for **targeted intervention** at high-impact sites.

These findings offer a clear direction for AEMR to **prioritize reliability investments**, conduct **performance audits**, or pursue **infrastructure upgrades** at the most vulnerable grid nodes.


In [14]:
%%sql

SELECT
  ROUND(AVG(Energy_Lost_MW), 2) AS Avg_Energy_Lost,
  ROUND(SUM(Energy_Lost_MW), 2) AS Total_Energy_Lost,
  ROUND(
    SUM(Energy_Lost_MW) * 100.0 /
    (SELECT SUM(Energy_Lost_MW)
     FROM AEMR_Outage_Table
     WHERE Status = 'Approved'
       AND Outage_Reason = 'Forced'
       AND Year = outer_tbl.Year),
    2
  ) AS Pct_Energy_Loss,
  Facility_Code,
  Participant_Code,
  Outage_Reason,
  Year
FROM AEMR_Outage_Table AS outer_tbl
WHERE Status = 'Approved'
  AND Outage_Reason = 'Forced'
GROUP BY Facility_Code, Participant_Code, Year
ORDER BY Total_Energy_Lost DESC, Year DESC;


 * sqlite:///temp_db_file.db
Done.


Avg_Energy_Lost,Total_Energy_Lost,Pct_Energy_Loss,Facility_Code,Participant_Code,Outage_Reason,Year
44.16,21639.55,26.39,AURICON_PNJ_U1,AURICON,Forced,2017
85.14,19326.56,23.57,BW1_GREENWATERS_G2,GW,Forced,2017
49.69,15751.38,22.41,BW1_GREENWATERS_G2,GW,Forced,2016
87.71,13771.07,19.59,MELK_G7,MELK,Forced,2016
51.42,10696.28,15.22,AURICON_PNJ_U1,AURICON,Forced,2016
58.11,10285.4,12.54,MELK_G7,MELK,Forced,2017
131.78,9093.09,12.93,PMC_AG,PMC,Forced,2016
72.61,5881.52,8.37,PJRH_GT11,PJRH,Forced,2016
141.21,5648.44,6.89,PMC_AG,PMC,Forced,2017
29.17,5016.67,6.12,TIWEST_COG1,TRMOS,Forced,2017


<h3 style="color:#D4AF37"> Business Question Ten </h3>

<b>What are the primary causes behind the highest energy losses for the top three participants (GW, MELK, and AURICON), and how much do these outage reasons contribute to their overall energy loss?



In [15]:
%%sql
SELECT
  Participant_Code,
  Facility_Code,
  Description_Of_Outage,
  ROUND(SUM(Energy_Lost_MW), 2) AS Total_Energy_Lost,
  ROUND(SUM(Energy_Lost_MW) * 100.0 /
        (SELECT SUM(Energy_Lost_MW)
         FROM AEMR_Outage_Table
         WHERE Status = 'Approved' AND Outage_Reason = 'Forced'), 2) AS Pct_Energy_Loss,
  RANK() OVER (PARTITION BY Participant_Code ORDER BY SUM(Energy_Lost_MW) DESC) AS rank
FROM AEMR_Outage_Table
WHERE Participant_Code IN ('GW', 'MELK', 'AURICON')
  AND Status = 'Approved'
  AND Outage_Reason = 'Forced'
GROUP BY Participant_Code, Facility_Code, Description_Of_Outage
ORDER BY Participant_Code, Total_Energy_Lost DESC;


 * sqlite:///temp_db_file.db
Done.


Participant_Code,Facility_Code,Description_Of_Outage,Total_Energy_Lost,Pct_Energy_Loss,rank
AURICON,AURICON_PNJ_U1,Full unit trip,6033.87,3.96,1
AURICON,AURICON_PNJ_U1,Return to service from outage delayed,1573.0,1.03,2
AURICON,AURICON_PNJ_U1,unit maximum output was restricted to 100 MW as-generated due to high rotor temperature alarm for safety purposes,1167.73,0.77,3
AURICON,AURICON_PNJ_U1,WGP U2 vibration issues during start-up sequence.,1140.0,0.75,4
AURICON,AURICON_PNJ_U1,Unit slow to ramp up.,676.94,0.44,5
AURICON,AURICON_PNJ_U1,Unit RTS & ramp-up delayed due to reset of overspeed tripping solenoid.,564.53,0.37,6
AURICON,AURICON_PNJ_U1,Unit trip,530.27,0.35,7
AURICON,AURICON_PNJ_U1,"Wagerup unit 2 trip, control fault",499.64,0.33,8
AURICON,AURICON_PNJ_U1,Pinjarra Unit 2 ramping up from an outage,472.19,0.31,9
AURICON,AURICON_PNJ_U1,Limited output due to ambient conditions (High temperature),460.4,0.3,10


**Insight**:

An in-depth analysis of forced outage data across 2016–2017 reveals that the top three contributors to energy loss—**GW**, **AURICON**, and **MELK**—each face distinct root causes behind their most severe disruptions:

- **GW (BW1_GREENWATERS_G2)** experienced the highest energy loss due to *real-time operational issues*, resulting in over **28,600 MWh lost**—accounting for nearly **19%** of all forced outage-related energy loss. This highlights significant gaps in operational readiness and system stability.

- **AURICON (AURICON_PNJ_U1)** saw its largest impact from a *full unit trip*, contributing **6,034 MWh** in lost energy, or nearly **4%** of the total. This suggests critical *equipment-level vulnerabilities* affecting supply continuity.

- **MELK (MELK_G7)** reported **1,100 MWh** in losses due to *safety-related shutdowns*. Although lower in volume, the nature of the issue raises concerns around *risk management* and *regulatory compliance*.
